In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2

import math

def grayscale(img):    
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
       
def convert_hls(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    
def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)   
    
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
            
    cv2.fillPoly(mask, vertices, ignore_mask_color)
        
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[0, 255, 0], thickness=2):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img


def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    return cv2.addWeighted(initial_img, α, img, β, λ)

slopeLeftStored = 0;
slopeRightStored = 0;
bLeftStored = 0;
bRightStored = 0;

def draw_lanes(image):
#    image = mpimg.imread(imageName)
    
    converted = convert_hls(image)
    # white color mask
    lower = np.uint8([  0, 200,   0])
    upper = np.uint8([255, 255, 255])
    white_mask = cv2.inRange(converted, lower, upper)
    # yellow color mask
    lower = np.uint8([ 10,   0, 100])
    upper = np.uint8([ 40, 255, 255])
    yellow_mask = cv2.inRange(converted, lower, upper)
    # combine the mask
    mask = cv2.bitwise_or(white_mask, yellow_mask)
    whiteYellowImage = cv2.bitwise_and(image, image, mask = mask)


    grayImage = grayscale(whiteYellowImage)
    blurredImage = gaussian_blur(grayImage, 5)
    edgesImage = canny(blurredImage, 40, 80)
    
    vertices = np.array([[0, image.shape[0]], [image.shape[1]*0.5,image.shape[0]*0.55], [image.shape[1],image.shape[0]]], np.int32)
    maskedImage = region_of_interest(edgesImage, [vertices])

    rho = 2
    theta = np.pi/180
    threshold = 20
    min_line_length = 20
    max_line_gap = 5

    lines = cv2.HoughLinesP(maskedImage, rho, theta, threshold, np.array([]), minLineLength=min_line_length, maxLineGap=max_line_gap)
    houghLinesImage = np.zeros((maskedImage.shape[0], maskedImage.shape[1], 3), dtype=np.uint8)
    draw_lines(houghLinesImage, lines)

    yPointsLeft = [];
    yPointsRight = [];
                 
    linesFiltered = [] 
   
    cumLengthLeft = 0
    cumLengthRight = 0
    aWeightedLeft = 0;
    aWeightedRight = 0;
    bWeightedLeft = 0;
    bWeightedRight = 0;
    
    linesParams = []
                
    for line in lines:
        for x1,y1,x2,y2 in line:
            a = float((y2-y1)/(x2-x1))
            b = (y1-a*x1)      
            length = math.sqrt(pow(y2-y1,2)+pow(x2-x1,2))
           
            if not np.isnan(a) or np.isinf(a) or (a == 0):
                if (a > -1.5) and (a < -0.3) :
                    linesFiltered.append(line) 
                    yPointsLeft.append(y1)
                    yPointsLeft.append(y2)
                    cumLengthLeft += pow(length,2)
                    aWeightedLeft += a * pow(length,2)
                    bWeightedLeft += b * pow(length,2)
                    
                if (a > 0.3) and (a < 1.5) :
                    linesFiltered.append(line)   
                    yPointsRight.append(y1)
                    yPointsRight.append(y2)
                    cumLengthRight += pow(length,2)
                    aWeightedRight += a * pow(length,2)
                    bWeightedRight += b * pow(length,2)
              
              
    houghLinesFilteredImage = np.zeros((maskedImage.shape[0], maskedImage.shape[1], 3), dtype=np.uint8)
    draw_lines(houghLinesFilteredImage, linesFiltered)
        
    x1_left=0
    x2_left=0
    x1_right=0
    x2_right=0
    a_left = 0
    b_left =0
    a_right=0
    b_right=0
    y_left_min=0
    y_right_min =0    
    y_min=0
    y_max = image.shape[0]
    
      
    if (len(yPointsLeft) > 0): 
        y_left_min=min(yPointsLeft)
    if (len(yPointsRight) > 0):     
        y_right_min=min(yPointsRight)
    if (y_left_min>0 and y_right_min>0):    
        y_min = min(y_left_min, y_right_min)
    elif (y_left_min>0 and y_right_min==0):
        y_min = y_left_min
    elif (y_left_min==0 and y_right_min>0):
        y_min = y_right_min
    
    if (len(yPointsLeft) > 0):                     
        aWeightedLeft /= cumLengthLeft    
        bWeightedLeft /= cumLengthLeft                
        a_left = aWeightedLeft
        b_left = bWeightedLeft
             
    if (len(yPointsRight) > 0):
        aWeightedRight /= cumLengthRight
        bWeightedRight /= cumLengthRight
        a_right = aWeightedRight
        b_right = bWeightedRight           
        
    yIntersection = int((a_right*b_left-a_left*b_right)/(a_right-a_left))
    margin = 10
    if (yIntersection + margin >y_min):   
        y_min = yIntersection + margin
    
    if (a_left!=0):    
        x1_left = int((y_max-b_left)/a_left)
        x2_left = int((y_min-b_left)/a_left)
    
    if (a_right!=0): 
        x1_right = int((y_max-b_right)/a_right)
        x2_right = int((y_min-b_right)/a_right)

    foundLinesImage = np.zeros((maskedImage.shape[0], maskedImage.shape[1], 3), dtype=np.uint8)
       
    if (a_left!=0):    
        cv2.line(foundLinesImage, (x1_left, y_max), (x2_left, y_min), [255, 0, 0], 7)
    
    if (a_right!=0): 
        cv2.line(foundLinesImage, (x1_right, y_max), (x2_right, y_min), [255, 0, 0], 7)
    
    origWithHoughLines = weighted_img(houghLinesImage,image)
    origWithHoughLinesFiltered = weighted_img(houghLinesFilteredImage,image)
    origWithFoundLanes = weighted_img(foundLinesImage,image)

   
#    plt.figure(figsize = (int(image.shape[1]/30),int(image.shape[0])/30))
#    plt.subplot(141)
#    plt.imshow(whiteYellowImage)
#    plt.subplot(142)
#    plt.imshow(grayImage, cmap='gray')
#    plt.subplot(143)
#    plt.imshow(blurredImage, cmap='gray')
#    
#    plt.figure(figsize = (int(image.shape[1]/30),int(image.shape[0])/30))
#    plt.subplot(141)
#    plt.imshow(maskedImage, cmap='gray')
#   # plt.figure()
#    plt.subplot(142)
#    plt.imshow(origWithHoughLines)
#    plt.subplot(143)
#    plt.imshow(origWithHoughLinesFiltered)
#    plt.subplot(144)
#    plt.imshow(origWithFoundLanes)

    return origWithFoundLanes
    


#import os

#draw_lanes("test_images/challengeSnap5.jpg")

#for img in os.listdir("test_images/"):
#   draw_lanes("test_images/"+img)
   


from moviepy.editor import VideoFileClip
from IPython.display import HTML

#solidWhiteRight_out = 'solidWhiteRight_out.mp4'
#clip1 = VideoFileClip("solidWhiteRight.mp4")

#clip = clip1.fl_image(draw_lanes)
#%time clip.write_videofile(solidWhiteRight_out, audio=False)

#solidYellowLeft_out = 'solidYellowLeft_out.mp4'
#clip1 = VideoFileClip("solidYellowLeft.mp4")

#clip = clip1.fl_image(draw_lanes)
#%time clip.write_videofile(solidYellowLeft_out, audio=False)


challenge_out = 'test_movies/challenge_out.mp4'
clip1 = VideoFileClip("test_movies/challenge.mp4")

clip = clip1.fl_image(draw_lanes)
%time clip.write_videofile(challenge_out, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_out))

